In [153]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
# load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
trueCountsDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/reads_fastq"

trueCountsDf <- lapply(seq(12), function(i) {
    samp <- ifelse(i<=6,i,i-6)
    rep <- ifelse(i<=6,1,2)
    file <- file.path(trueCountsDir, samp, paste(paste("true_counts", rep,sep="_"),"tsv",sep="."))
    df <- read.delim(file, row.names = 1)
})

comp_mard <- function(x, y, cutoff = 0.01) {
    if(length(x) != length(y))
        stop(paste(length(x), length(y)))
    relDiff <- rep(0, length(x))
    zInds <- which(x <= cutoff & y <= cutoff)
#     print(length(zInds)/length(x))
    nzInds <- setdiff(seq_along(x), zInds)
    relDiff[nzInds] <- abs(x[nzInds] - y[nzInds])/(x[nzInds] + y[nzInds])
    return(sum(relDiff)/length(x))
}

In [154]:
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeThr.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons0.RData")

load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/cutsDepth.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/updated/comp_trees/lfcCuts_len.RData")

In [155]:
dfCor <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfCor) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfMard <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfMard) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

In [156]:
dfCorCuts <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfCorCuts) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")

dfMardCuts <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfMardCuts) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")

#### Consensus

In [157]:
consTxps <- treeCons$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(consTxps, trueTxps)
mb <- mergeLeaves(treeCons, seBrainSim[allTxps,])
yCons <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCons <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCons)[["counts"]])

[1] "Missing txps 726"


In [158]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [159]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7
dfCor$Sample <- colnames(seBrainSim)
dfCor$Type <- rep("Consensus", ncol(seBrainSim))
dfMard$Sample <- colnames(seBrainSim)
dfMard$Type <- rep("Consensus", ncol(seBrainSim))

# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCons[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfCor[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggCons[depth==i,j]))
    dfMard[[n]] <- mat[i,]
}



In [160]:
dfCorCuts$Sample <- colnames(seBrainSim)
dfCorCuts$Type <- rep("Consensus", ncol(seBrainSim))
dfMardCuts$Sample <- colnames(seBrainSim)
dfMardCuts$Type <- rep("Consensus", ncol(seBrainSim))
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=4)
for(i in seq_along(dCutsD)) {
    dd <- dCutsD[[i]][["Cons"]]
    iCuts <- dd[dd > length(treeCons$tip)]
    iCuts <- iCuts + length(tr$tip)-length(treeCons$tip)
    leaves <- match(treeCons$tip[dd[dd <= length(treeCons$tip)]], tr$tip)
    dd <- c(leaves, iCuts)
    n <- paste("Gamma",c(0.1, 0.5, 1, 5)[i], sep="_")
    mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggCons[dd,],method="spearman"))
    dfCorCuts[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggCons[dd,j]))
    dfMardCuts[[n]] <- mat[i,]
}

dd <- dCuts[["Consensus"]]
iCuts <- dd[dd > length(treeCons$tip)]
iCuts <- iCuts + length(tr$tip)-length(treeCons$tip)
leaves <- match(treeCons$tip[dd[dd <= length(treeCons$tip)]], tr$tip)
dd <- c(leaves, iCuts)
n <- "LFC"
mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggCons[dd,],method="spearman"))
dfCorCuts[[n]] <- mat[i,]
mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggCons[dd,j]))
dfMardCuts[[n]] <- mat[i,]

In [161]:
dfCorCuts

Sample,Gamma_0.1,Gamma_0.5,Gamma_1,Gamma_5,LFC,Type
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1_1,0.9872721,0.9719999,0.9660248,0.9533866,0.9949299,Consensus
2_1,0.9874307,0.9725895,0.9668935,0.9534542,0.9949068,Consensus
3_1,0.9884198,0.9748012,0.9695085,0.9582890,0.9954835,Consensus
4_1,0.9877949,0.9726840,0.9668950,0.9545140,0.9953477,Consensus
5_1,0.9880292,0.9749937,0.9699611,0.9593492,0.9954616,Consensus
6_1,0.9883036,0.9750716,0.9701320,0.9592553,0.9955019,Consensus
1_2,0.9879183,0.9726345,0.9664461,0.9523791,0.9952344,Consensus
2_2,0.9874604,0.9723130,0.9663612,0.9535847,0.9948480,Consensus
3_2,0.9886385,0.9745664,0.9689979,0.9571962,0.9954964,Consensus


#### Correlation Tree

In [162]:
corTxps <- treeCor$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(corTxps, trueTxps)
mb <- mergeLeaves(treeCor, seBrainSim[allTxps,])
yCor <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCor <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCor)[["counts"]])

[1] "Missing txps 726"


In [163]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [164]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Anti-Correlation", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Anti-Correlation", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCor[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggCor[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}
dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


In [165]:
dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT2) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Anti-Correlation", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Anti-Correlation", ncol(seBrainSim))
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=4)
for(i in seq_along(dCutsD)) {
    dd <- dCutsD[[i]][["Cor"]]
    iCuts <- dd[dd > length(treeCons$tip)]
    iCuts <- iCuts + length(tr$tip)-length(treeCor$tip)
    leaves <- match(treeCor$tip[dd[dd <= length(treeCor$tip)]], tr$tip)
    dd <- c(leaves, iCuts)
    n <- paste("Gamma",c(0.1, 0.5, 1, 5)[i], sep="_")
    mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggCor[dd,],method="spearman"))
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggCor[dd,j]))
    dfT2[[n]] <- mat[i,]
}

dd <- dCuts[["Anti-Correlation"]]
iCuts <- dd[dd > length(treeCor$tip)]
iCuts <- iCuts + length(tr$tip)-length(treeCor$tip)
leaves <- match(treeCor$tip[dd[dd <= length(treeCor$tip)]], tr$tip)
dd <- c(leaves, iCuts)
n <- "LFC"
mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggCor[dd,],method="spearman"))
dfT[[n]] <- mat[i,]
mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggCor[dd,j]))
dfT2[[n]] <- mat[i,]

dfCorCuts <- rbind(dfCorCuts, dfT)
dfMardCuts <- rbind(dfMardCuts, dfT2)


#### Mean Consensus

In [166]:
miTxps <- treeMeanInf$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(miTxps, trueTxps)
mb <- mergeLeaves(treeMeanInf, seBrainSim[allTxps,])
yMI <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggMI <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yMI)[["counts"]])

[1] "Missing txps 726"


In [167]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [168]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf", ncol(seBrainSim))

# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggMI[depth==i,j]))
    dfT2[[n]] <- mat[i,]
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI[depth==i,]))

}

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


In [169]:
dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT2) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf", ncol(seBrainSim))
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=4)
for(i in seq_along(dCutsD)) {
    dd <- dCutsD[[i]][["MeanInf"]]
    iCuts <- dd[dd > length(treeMeanInf$tip)]
    iCuts <- iCuts + length(tr$tip)-length(treeMeanInf$tip)
    leaves <- match(treeMeanInf$tip[dd[dd <= length(treeMeanInf$tip)]], tr$tip)
    dd <- c(leaves, iCuts)
    n <- paste("Gamma",c(0.1, 0.5, 1, 5)[i], sep="_")
    mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggMI[dd,],method="spearman"))
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggMI[dd,j]))
    dfT2[[n]] <- mat[i,]
}

dd <- dCuts[["MeanInf"]]
iCuts <- dd[dd > length(treeCor$tip)]
iCuts <- iCuts + length(tr$tip)-length(treeCor$tip)
leaves <- match(treeMeanInf$tip[dd[dd <= length(treeCor$tip)]], tr$tip)
dd <- c(leaves, iCuts)
n <- "LFC"
mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggMI[dd,],method="spearman"))
dfT[[n]] <- mat[i,]
mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggMI[dd,j]))
dfT2[[n]] <- mat[i,]

dfCorCuts <- rbind(dfCorCuts, dfT)
dfMardCuts <- rbind(dfMardCuts, dfT2)


#### MeanInf0

In [170]:
mi0Txps <- treeMeanInf0$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(mi0Txps, trueTxps)
mb <- mergeLeaves(treeMeanInf0, seBrainSim[allTxps,])
yMI0 <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggMI0 <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yMI0)[["counts"]])

[1] "Missing txps 726"


In [171]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [172]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf0", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf0", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI0[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggMI0[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}
dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


In [173]:
dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT2) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf0", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf0", ncol(seBrainSim))
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=4)
for(i in seq_along(dCutsD)) {
    dd <- dCutsD[[i]][["MeanInf0"]]
    iCuts <- dd[dd > length(treeMeanInf$tip)]
    iCuts <- iCuts + length(tr$tip)-length(treeMeanInf$tip)
    leaves <- match(treeMeanInf0$tip[dd[dd <= length(treeMeanInf0$tip)]], tr$tip)
    dd <- c(leaves, iCuts)
    n <- paste("Gamma",c(0.1, 0.5, 1, 5)[i], sep="_")
    mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggMI0[dd,],method="spearman"))
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggMI0[dd,j]))
    dfT2[[n]] <- mat[i,]
}

dd <- dCuts[["MeanInf0"]]
iCuts <- dd[dd > length(treeCor$tip)]
iCuts <- iCuts + length(tr$tip)-length(treeCor$tip)
leaves <- match(treeMeanInf0$tip[dd[dd <= length(treeMeanInf0$tip)]], tr$tip)
dd <- c(leaves, iCuts)
n <- "LFC"
mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggMI0[dd,],method="spearman"))
dfT[[n]] <- mat[i,]
mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggMI0[dd,j]))
dfT2[[n]] <- mat[i,]

dfCorCuts <- rbind(dfCorCuts, dfT)
dfMardCuts <- rbind(dfMardCuts, dfT2)


In [174]:
sum(treeCons0$tip.label ==treeMeanInf$tip.label)

[1] 0

#### Thr Terminus

In [175]:
thrTxps <- treeThr$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(thrTxps, trueTxps)
mb <- mergeLeaves(treeThr, seBrainSim[allTxps,])
yThr <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggThr <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yThr)[["counts"]])

[1] "Missing txps 726"


In [176]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [177]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Thr", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Thr", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggThr[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggThr[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}
dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


#### Consensus0

In [178]:
cons0Txps <- treeCons0$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(cons0Txps, trueTxps)
mb <- mergeLeaves(treeCons0, seBrainSim[allTxps,])
yCons0 <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCons0 <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCons0)[["counts"]])

[1] "Missing txps 726"


In [179]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [180]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Consensus0", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Consensus0", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCons0[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggCons0[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}
dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


In [181]:
dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=7))
colnames(dfT2) <- c("Sample", c(paste("Gamma",c(0.1, 0.5, 1, 5), sep="_"), "LFC"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Consensus0", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Consensus0", ncol(seBrainSim))
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=4)
for(i in seq_along(dCutsD)) {
    dd <- dCutsD[[i]][["Cons0"]]
    iCuts <- dd[dd > length(treeMeanInf$tip)]
    iCuts <- iCuts + length(tr$tip)-length(treeMeanInf$tip)
    leaves <- match(treeCons0$tip[dd[dd <= length(treeCons0$tip)]], tr$tip)
    dd <- c(leaves, iCuts)
    n <- paste("Gamma",c(0.1, 0.5, 1, 5)[i], sep="_")
    mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggCons0[dd,],method="spearman"))
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggCons0[dd,j]))
    dfT2[[n]] <- mat[i,]
}

dd <- dCuts[["Consensus0"]]
iCuts <- dd[dd > length(treeCor$tip)]
iCuts <- iCuts + length(tr$tip)-length(treeCor$tip)
leaves <- match(treeCons0$tip[dd[dd <= length(treeCons0$tip)]], tr$tip)
dd <- c(leaves, iCuts)
n <- "LFC"
mat[i,] <- diag(cor(trueCountsAgg[dd,], yAggCons0[dd,],method="spearman"))
dfT[[n]] <- mat[i,]
mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[dd,j], yAggCons0[dd,j]))
dfT2[[n]] <- mat[i,]

dfCorCuts <- rbind(dfCorCuts, dfT)
dfMardCuts <- rbind(dfMardCuts, dfT2)


In [182]:
dfCor %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))
dfMard %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.9446778,0.9913943,0.9977744,0.9994178,0.9997756,0.9998456,0.9999731
Consensus,0.9446778,0.9910081,0.9955883,0.9976570,0.9986886,0.9992657,0.9997477
Consensus0,0.9446778,0.9982314,0.9996337,0.9998230,0.9998939,0.9999222,0.9999778
Mean Inf,0.9446778,0.9976634,0.9991590,0.9994891,0.9996490,0.9997309,0.9999152
Mean Inf0,0.9446778,0.9986109,0.9996749,0.9998361,0.9998941,0.9999174,0.9999696
Thr,0.9446778,0.9975392,0.9990993,0.9994557,0.9996700,0.9997474,0.9996629


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.09512794,0.06844116,0.02929967,0.013500659,0.006791058,0.004010136,0.002000134
Consensus,0.09512794,0.06253313,0.05639070,0.048312455,0.040397517,0.032071083,0.017485159
Consensus0,0.09512794,0.02979510,0.01372657,0.008009757,0.004917904,0.003577308,0.001467272
Mean Inf,0.09512794,0.03485119,0.02631151,0.022779845,0.019636236,0.017965069,0.010082454
Mean Inf0,0.09512794,0.02518964,0.01103742,0.006958612,0.004819988,0.003715719,0.001796846
Thr,0.09512794,0.04523547,0.01965747,0.012440287,0.007716122,0.005357956,0.005741053


In [183]:
dfCorCuts %>% group_by(Type) %>% select(starts_with("Gamma"), "LFC") %>% summarise_all(.funs = c(mean="mean"))
dfMardCuts %>% group_by(Type) %>% select(starts_with("Gamma"), "LFC") %>% summarise_all(.funs = c(mean="mean"))


Adding missing grouping variables: `Type`


Type,Gamma_0.1_mean,Gamma_0.5_mean,Gamma_1_mean,Gamma_5_mean,LFC_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.9852277,0.9741371,0.9680465,0.9554401,0.9667377
Consensus,0.9880326,0.9738201,0.9682663,0.9561663,0.9953219
Consensus0,0.9861347,0.9739568,0.9684485,0.9562235,0.9980551
Mean Inf,0.9848260,0.9733446,0.9681640,0.9561241,0.9921035
Mean Inf0,0.9855208,0.9734543,0.9681711,0.9561243,0.9984294


Adding missing grouping variables: `Type`


Type,Gamma_0.1_mean,Gamma_0.5_mean,Gamma_1_mean,Gamma_5_mean,LFC_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.01390588,0.04785584,0.06138730,0.08499604,0.02554406
Consensus,0.02933941,0.05457730,0.06476810,0.08471748,0.02414488
Consensus0,0.03056762,0.05379882,0.06419443,0.08468519,0.01798458
Mean Inf,0.03502843,0.05479817,0.06468651,0.08474657,0.02954133
Mean Inf0,0.03191872,0.05462998,0.06467484,0.08474499,0.01727838
